# 5.1 层和块


In [5]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=10)
])

X = tf.random.uniform(shape=(2, 20))
net(X)


ERROR! Session/line number was not unique in database. History logging moved to new session 349


<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.4384773 , -0.1545909 ,  0.29013187,  0.02995207, -0.12894422,
        -0.04671464,  0.3074219 ,  0.11242911, -0.03223428, -0.40392643],
       [ 0.22151658,  0.0116825 ,  0.22800471, -0.13516878, -0.03865247,
        -0.09460817,  0.10633341,  0.19312803, -0.09057835, -0.17588806]],
      dtype=float32)>

In [6]:
import tensorflow as tf

# 自定义块
class MLP(tf.keras.Model):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs, training=None, mask=None):
        return self.out(self.hidden(inputs))

X = tf.random.uniform(shape=(2, 20))
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.23698121, -0.2666634 , -0.41143692, -0.02265769, -0.12346094,
        -0.25443998, -0.35757905, -0.06618749,  0.14136459,  0.06184211],
       [-0.3148666 , -0.07989764, -0.32108575,  0.15359978, -0.24419282,
        -0.25240648, -0.39858675,  0.35415533,  0.05973592, -0.2504194 ]],
      dtype=float32)>

In [ ]:
import tensorflow as tf

# 顺序块
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super(MySequential, self).__init__()
        self.modules = []
        for module in args:
            self.modules.append(module)

    def call(self, inputs, training=None, mask=None):
        for module in self.modules:
            inputs = module(inputs)
        return inputs


X = tf.random.uniform(shape=(2, 20))
net = MySequential(tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
                   tf.keras.layers.Dense(units=10))
net(X)



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\DEVTOOLS\Anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-9119f6623fe3>", line 20, in <module>
    net(X)
  File "D:\DEVTOOLS\Anaconda3\envs\tf\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py", line 822, in __call__
    outputs = self.call(cast_inputs, *args, **kwargs)
  File "<ipython-input-7-9119f6623fe3>", line 13, in call
    X = module(X)
UnboundLocalError: local variable 'X' referenced before assignment

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\DEVTOOLS\Anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'UnboundLocalError' object has no attribute '_render_traceback_'

During handling of the above exception, another except

In [8]:
# 前向传播中执行代码
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super(FixedHiddenMLP, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(tf.random.uniform(shape=(20, 20)))
        self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)

    def call(self, inputs, training=None, mask=None):
        x = self.flatten(inputs)
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)  # 1: bias
        x = self.dense(x)
        while tf.reduce_sum(tf.math.abs(x)) > 1:
            x /= 2
        return tf.reduce_sum(x)

net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7084787>

In [15]:
# 5.2 参数管理
import tensorflow as tf

net = tf.keras.Sequential([tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(units=4, activation=tf.nn.relu),
                           tf.keras.layers.Dense(units=1)])
X = tf.random.uniform(shape=(2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[ 0.03131925],
       [-0.06229562]], dtype=float32)>

In [16]:
print(net.layers[2].weights)

[<tf.Variable 'sequential_5/dense_34/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.5606878 ],
       [ 0.09798849],
       [-0.5030795 ],
       [ 0.35627353]], dtype=float32)>, <tf.Variable 'sequential_5/dense_34/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [20]:
# 目标参数
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(value=net.layers[2].weights[1]))
print(tf.convert_to_tensor(value=net.layers[2].weights[1]).numpy().tolist())


<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'sequential_5/dense_34/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)
[0.0]


In [23]:
# 一次性访问所有参数
print(net.get_weights()[0])  # W
print(net.get_weights()[1])  # bias


[[-0.14269078 -0.61519194  0.34146804 -0.23348016]
 [-0.03623539 -0.42631355 -0.6627742   0.7363996 ]
 [ 0.05565244  0.7090532   0.30032223 -0.7025544 ]
 [ 0.13628417 -0.5572754   0.6095025   0.22789472]]
[0. 0. 0. 0.]


In [24]:
# 从嵌套层中收集参数
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=4, activation=tf.nn.relu)
    ], name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        net.add(block1(name=f'block-{i}'))
    return net

rgret = tf.keras.Sequential()
rgret.add(block2())
rgret.add(tf.keras.layers.Dense(1))
rgret(X)


<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[ 0.        ],
       [-0.11435951]], dtype=float32)>

In [28]:
print(rgret.summary())
print(rgret.layers[0].summary())
print(rgret.layers[0].layers[1].layers[1].weights[0])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_7 (Sequential)    multiple                  80        
_________________________________________________________________
dense_39 (Dense)             multiple                  5         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block-0 (Sequential)         multiple                  20        
_________________________________________________________________
block-1 (Sequential)         multiple                  20        
_________________________________________________________________
block-2 (Sequential)         multiple                  20        
_______________________________

In [29]:
# 内置初始化
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=4, activation=tf.nn.relu,
                          kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
                          bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(units=1)

])

net(X)
print(net.weights[0], net.weights[1])


<tf.Variable 'sequential_8/dense_40/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.00951986, -0.00250946, -0.00323118, -0.00305062],
       [-0.00659536,  0.01299575, -0.01094384,  0.00433804],
       [ 0.00831138, -0.00495221, -0.00573723, -0.00513852],
       [-0.00379328, -0.01355408, -0.01401954,  0.00673643]],
      dtype=float32)> <tf.Variable 'sequential_8/dense_40/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>


In [30]:
# 内置初始化-常量初始化
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=4, activation=tf.nn.relu,
                          kernel_initializer=tf.constant_initializer(value=1),
                          bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(units=1)

])

net(X)
print(net.weights[0], net.weights[1])



<tf.Variable 'sequential_9/dense_42/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)> <tf.Variable 'sequential_9/dense_42/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>
